# CryoSat-2 Neural Network

## 19th Sept 2018
### Claude de Rijke-Thomas

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import csv

In [5]:
#reading the csv data as a data frame:
data_frame = pd.read_csv('Echo_Characteristics.csv')
all_classes = data_frame['Class'].values
all_features = data_frame.drop('Class', axis=1).values

one_hot_flat = []
for label in all_classes:
    if label =='lead':
        one_hot_flat.extend([1,0,0])
    if label =='ice':
        one_hot_flat.extend([0,1,0])
    if label=='ocean':
        one_hot_flat.extend([0,0,1])
print(np.shape(one_hot_flat)[0])
#making an array of shape [[, , ],[, , ], ...]
one_hot_labels = np.reshape(one_hot_flat, (int(round(np.shape(one_hot_flat)[0]/3)), 3)) 
print(np.shape(one_hot_labels))
print(one_hot_labels[4000:4005])



14580
(4860, 3)
[[0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [6]:
def create_model():
    model = Sequential()
    #adding 400 nodes to the shallow layer:
    model.add(Dense(400, input_dim=int(5), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    # Another hidden layer of 16 units
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    #output layer
    model.add(Dense(3, kernel_initializer='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
# Wrap our Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=30, verbose=0)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, all_features, one_hot_labels, cv=15) # cv = train test split (k-1: 1)
cv_scores.mean()



0.9226337448559672

In [ ]:
#Shallow (400 nodes), 'relu', 20% dropout, activation = softmax, optimizer = adam, cv = 10 : 0.889094650156704
#deep1 (400 nodes, 16 nodes), 'relu', 20% dropout, activation = softmax, optimizer = adam, cv = 10 : 0.913168724230778
##deep2 (400 nodes, 16 nodes, 16 nodes), """": 0.8946502056877309
#deep1 (400 nodes, 16 nodes), 'relu', 20% dropout, activation = softmax, optimizer = adam, cv = 5 : 0.7382716049934611
# " cv = 15: 0.9141975308641975
# " 0.9265432098765433